This notebook demonstrates a basic SQL agent that translates natural language questions into SQL queries.

In [ ]:
import json
import os
from typing import Annotated, Dict

from autogen import ConversableAgent, UserProxyAgent, ConversationPattern, AssistantAgent, config_list_from_json
from dotenv import load_dotenv
import autogen
from typing import Annotated
from openai import AzureOpenAI
import sqlite3
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.utilities.sql_database import SQLDatabase
from langchain.llms import OpenAI

load_dotenv()
AZURE_OPENAI_ENDPOINT=os.getenv("AISTUDIO_AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_GPT4o_DEPLOYMENT=os.getenv("AI_STUDIO_AZURE_OPENAI_GPT4o_DEPLOYMENT")
AZURE_OPENAI_API_VERSION="2024-02-01"
AZURE_OPENAI_KEY=os.getenv("AISTUDIO_AZURE_OPENAI_KEY")
OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")


In [3]:
llm = AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_KEY,
        api_version=AZURE_OPENAI_API_VERSION
)

## Agent Implementation

Using AutoGen, a SQL agent can be implemented with a ConversableAgent. It executes the generated SQL query and the agent can take execution results as feedback to improve its generation in multiple rounds of conversations.

In [4]:
llm_config = {
    "cache_seed": 43,  # change the cache_seed for different trials
    "temperature": 0,
    "timeout": 120,  # in seconds
    "config_list": 
    [
        {
            "model": AZURE_OPENAI_GPT4o_DEPLOYMENT,
            "api_type": "azure",
            "api_key": AZURE_OPENAI_KEY,
            "base_url": AZURE_OPENAI_ENDPOINT,
            "api_version": AZURE_OPENAI_API_VERSION
        }
    ]
}


def check_termination(msg: Dict):
    if "tool_responses" not in msg:
        return False
    json_str = msg["tool_responses"][0]["content"]
    obj = json.loads(json_str)
    return "error" not in obj or obj["error"] is None and obj["reward"] == 1

sql_writer = ConversableAgent(
    name="sql_writer",
    llm_config=llm_config,
    system_message="You are good at writing SQL queries. Always respond with a function call to execute_sql().",
    is_termination_msg=check_termination,
    human_input_mode="NEVER",
)

assistant = autogen.AssistantAgent(
    name="assistant",
    system_message="For coding tasks, only use the functions you have been provided with. Reply in natural language to the questions asked. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
    is_termination_msg=check_termination,
    human_input_mode="NEVER",
)

user_proxy = UserProxyAgent("user_proxy", human_input_mode="NEVER", max_consecutive_auto_reply=5, code_execution_config={"work_dir":"coding", "use_docker":False})

@sql_writer.register_for_llm(description="Function for executing SQL query and returning a response")
@user_proxy.register_for_execution()
def execute_sql(sql: Annotated[str, "SQL query"]) -> Annotated[str, "results"]:
    database = 'bookstore.db'

    conn = sqlite3.connect(database)
    cur = conn.cursor()

    try:
        cur.execute(sql)
        rows = cur.fetchall()
        return rows
    finally:
        cur.close()
        conn.close()

group_chat = autogen.GroupChat(
    agents=[user_proxy, sql_writer], messages=[], max_round=5
)
manager = autogen.GroupChatManager(groupchat=group_chat, llm_config=llm_config)

# # Define a conversation pattern
# conversation_pattern = ConversationPattern(
#     agents=[hr_agent, tech_agent],
#     interactions=[
#         (hr_agent, "Ask about previous experience."),
#         (tech_agent, "Inquire about technical skills."),
#     ],
# )

# # Apply the pattern to your assistant agent
# assistant.apply_conversation_pattern(conversation_pattern)

The agent can then take as input the schema and the text question, and generate the SQL query.

In [5]:
question = "How many books are in the bookstore?"
message = f"""
    Answer the following question:
    {question}
"""

user_proxy.initiate_chat(
    sql_writer,
    message=message,
    llm_config=llm_config,
)

user_proxy (to sql_writer):


    Answer the following question:
    How many books are in the bookstore?


--------------------------------------------------------------------------------
sql_writer (to user_proxy):

***** Suggested tool call (call_E4gTkbfrf5OAo78CqQxX9RqY): execute_sql *****
Arguments: 
{"sql":"SELECT COUNT(*) FROM books;"}
****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION execute_sql...
user_proxy (to sql_writer):

***** Response from calling tool (call_E4gTkbfrf5OAo78CqQxX9RqY) *****
[[4]]
**********************************************************************

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': '\n    Answer the following question:\n    How many books are in the bookstore?\n', 'role': 'assistant', 'name': 'user_proxy'}, {'tool_calls': [{'id': 'call_E4gTkbfrf5OAo78CqQxX9RqY', 'function': {'arguments': '{"sql":"SELECT COUNT(*) FROM books;"}', 'name': 'execute_sql'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': '[[4]]', 'tool_responses': [{'tool_call_id': 'call_E4gTkbfrf5OAo78CqQxX9RqY', 'role': 'tool', 'content': '[[4]]'}], 'role': 'tool', 'name': 'user_proxy'}], summary='[[4]]', cost={'usage_including_cached_inference': {'total_cost': 0.000725, 'gpt-4o-2024-05-13': {'cost': 0.000725, 'prompt_tokens': 88, 'completion_tokens': 19, 'total_tokens': 107}}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])